<a href="https://colab.research.google.com/github/uwidger/eh/blob/master/NLTK_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd 
import gzip
import gensim 
import logging
from gensim.test.utils import common_texts
from gensim.models import word2vec, FastText, doc2vec


In [9]:
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import multiprocessing
import nltk
nltk.download('stopwords')
  
df = pd.read_csv('tcc_ceds_music.csv')

X = pd.DataFrame(df, columns=['lyrics'])    #Attempt to remove duplicates
for i in X:
  words = i.split()
  i = " ".join(sorted(set(words), key=words.index))

def clean_data(text):                       #Cleans data (removes unnecessary characters/symbols)
    text = re.sub(r'[^ \nA-Za-z0-9À-ÖØ-öø-ÿ/]+', '', text)
    text = re.sub(r'[\\/×\^\]\[÷]', '', text)
    return text
def change_lower(text):                      #Changes to lowercase            
    text = text.lower()
    return text
  
stopwords_list = stopwords.words("english")   #Commonly used words

def remover(text):                            #Gets rid of stopwords
    text_tokens = text.split(" ")
    final_list = [word for word in text_tokens if not word in stopwords_list]
    text = ' '.join(final_list)
    return text

def get_w2vdf(df):                            #Splits
    w2v_df = pd.DataFrame(df["lyrics"]).values.tolist()
    for i in range(len(w2v_df)):
        w2v_df[i] = w2v_df[i][0].split(" ")
    return w2v_df

def train_w2v(w2v_df):                        #Trains
    cores = multiprocessing.cpu_count()
    w2v_model = Word2Vec(min_count=4,
                         window=4,
                         size=300, 
                         alpha=0.03, 
                         min_alpha=0.0007, 
                         sg = 1,            #Using skip-gram algorithm
                         workers=cores-1)
    
    w2v_model.build_vocab(w2v_df, progress_per=10000)
    w2v_model.train(w2v_df, total_examples=w2v_model.corpus_count, epochs=100, report_delay=1)
    return w2v_model

df[["lyrics"]] = df[["lyrics"]].astype(str)       #Applying definitons
df["lyrics"] = df["lyrics"].apply(change_lower)
df["lyrics"] = df["lyrics"].apply(clean_data)
df["lyrics"] = df["lyrics"].apply(remover)
w2v_df = get_w2vdf(df)
w2v_model = train_w2v(w2v_df)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
X = pd.DataFrame(df, columns=['lyrics'])
y = df['genre']
y = y.replace('pop', 0)
y = y.replace('country', 1)
y = y.replace('blues', 2)
y = y.replace('jazz', 3)
y = y.replace('reggae', 4)
y = y.replace('rock', 5)
y = y.replace('hip hop', 6)

data = []

for idx,i in enumerate(X["lyrics"]):
  data.append([i, y[idx]])
  
  
df2=pd.DataFrame(data, columns=["lyrics", "genre"])

for idx, i in enumerate(df2["lyrics"]):
  df2.append([i, w2v_df[idx]])

  if  not w2v_df[idx]== float:
    df2 = df2[:-1]


#df2['lyrics'] = pd.to_numeric(df2['lyrics'], errors = 'coerce')
#df2.dropna(inplace = True)
df2.apply(pd.to_numeric, errors = 'coerce').dropna()     #Attempting to drop problematic rows in the datset, does drop them but will then assign new problematic rows 

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.4,random_state=42, stratify=y)

print( "Training cases: ", X_train.shape[0])
print("Test cases: " , X_test.shape[0])

#set regularization rate
reg=0.1


model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)    #Using logistic regression to classify
print(model)

predictions = model.predict(X_test)
print("Predicted labels: ", predictions)
print("Actual labels: ", y_test)

Training cases:  17022
Test cases:  11348


ValueError: ignored